In [20]:
import time
import sys
import os
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import pandas as pd
import cantera as ct

print(f"Running Cantera version: {ct.__version__}")
# Setting model and data directories.
SCRIPT_DIR  = os.getcwd()
MODEL_DIR   = os.path.join(SCRIPT_DIR, "data", "models")

Running Cantera version: 3.1.0


In [21]:
# We need to define the inlet conditions
reactor_temperature = 750  # Kelvin
reactor_pressure =  24 * ct.one_atm  # in atm. This equals 1.06 bars

# Selecting the mechanism.
mechString = 'aramco3.yaml'
mech = os.path.join(MODEL_DIR, mechString)

In [22]:
# This cell creates the reactor network for simulation.
gas = ct.Solution(mech)

gas.set_equivalence_ratio(phi=0.4875, fuel={'CH4':1.0}, oxidizer={'O2':1.0, 'N2':3.76})
gas.TP = reactor_temperature, reactor_pressure

residence_time = 2.1
reactor_volume = 1.96e-4

fuel_air_mixture_tank = ct.Reservoir(gas)
exhaust = ct.Reservoir(gas)

stirred_reactor = ct.IdealGasMoleReactor(gas, energy="off", volume=reactor_volume)

mass_flow_controller = ct.MassFlowController(
    upstream=fuel_air_mixture_tank,
    downstream=stirred_reactor,
    mdot=lambda t: stirred_reactor.mass / residence_time,
)

pressure_regulator = ct.PressureController(
    upstream=stirred_reactor,
    downstream=exhaust,
    primary=mass_flow_controller,
    K=1e-3,
    )

reactor_network = ct.ReactorNet([stirred_reactor])

In [23]:
# Create a SolutionArray to store the data
time_history = ct.SolutionArray(gas, extra=["t"])
max_simulation_time = 50  # seconds

tic = time.time()

# Set simulation start time to zero
t = 0
counter = 1

while t < max_simulation_time:
    t = reactor_network.step()
    if counter % 10 == 0:
        time_history.append(stirred_reactor.thermo.state, t=t)

    counter += 1

# Stop the stopwatch
toc = time.time()

print(f"Simulation Took {toc-tic:3.2f}s to compute, with {counter} steps")

species_to_report:list = ['CO', 'CH4']    # List containing species to report (mole fraction)

for i in species_to_report:
    x_species = time_history(i).X[-1]
    print(f"{i} is {x_species[0]:.8f}")

Simulation Took 0.44s to compute, with 448 steps
CO is 0.00405949
CH4 is 0.04305984


In [24]:
gas = ct.Solution('aramco3.yaml', transport_model=None)
equiv_ratio = 0.46 # lean combustion
gas.TP = 750, 24 * ct.one_atm
gas.set_equivalence_ratio(equiv_ratio, 'CH4:1.0', 'O2:1.0, N2:3.76')


inlet = ct.Reservoir(gas)
exhaust = ct.Reservoir(gas)

gas.equilibrate('HP')
combustor = ct.IdealGasMoleReactor(gas)
combustor.volume = reactor_volume

density = combustor.thermo.density
t_res = (reactor_volume * density) / 3.5

print(f"Residence time for the given combustor is {t_res:.8f} seconds | {t_res*1e+3:.8f} milliseconds.")

Residence time for the given combustor is 0.00026079 seconds | 0.26078660 milliseconds.
